In [2]:
from pytorchvideo.data.encoded_video import EncodedVideo

def read_gif(gif_path):
    """read gif and return dictionary as key ['video'] and value the tensor of size(CxTxHXW)"""

    video = EncodedVideo.from_path(gif_path)
    video = video.get_clip(0, 5) # get_clip fetches the clip from starting time to ending time

    return video

In [3]:
gif_folder_path = '/workspace/data/data_folder/o2o/gifs_11'
from glob import glob as glob

gif_paths = glob(gif_folder_path + '/*.gif')

In [4]:
issues = {}
issues['paths'] = []
issues['exceptions'] = []

from tqdm import tqdm as tqdm

for gif in tqdm(gif_paths):
    try:
        read_gif(gif)
    except Exception as e:
        issues['paths'].append(gif)
        issues['exceptions'].append(e)
        

100%|██████████| 2056/2056 [05:06<00:00,  6.71it/s]


In [8]:
def extract_gif(frame_index, yt_id, youtube_folder, output_folder, window_size):
    
    import cv2
    import os
    
    video_loc = os.path.join(youtube_folder, yt_id + '.mp4')
    vid = cv2.VideoCapture(video_loc)
    
    frame_count = int(vid.get(cv2.CAP_PROP_FRAME_COUNT))

    initial = frame_index - window_size
    final = frame_index + window_size

    if initial<0 or final>=frame_count:
        print("Issue for ", frame_index, " ",yt_id)
        return 0

    frames = []

    for i in range(frame_count):

        success, frame = vid.read()
        if i >= initial and i <= final and success:
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frames.append(rgb_frame)

    filename = yt_id + '_' + str(frame_index) + '_' + str(window_size) + '.gif'
    saving_location = os.path.join(output_folder, filename)

    import imageio
    fps = 4
    imageio.mimsave( saving_location, frames, fps=4)
    del vid
    return 1


# frame_index, yt_id, youtube_folder, output_folder, window_size

output_folder = '/workspace/data/data_folder/o2o/gifs_11'
window_size = 5
youtube_folder = '/workspace/data/data_folder/o2o/youtube2'


for path in issues['paths']:
    last_string = path.split('/')[-1]
    last_string = last_string.split('.')[0]
    yt_id, frame_index, _ = last_string.split('_')
    extract_gif( int(frame_index), yt_id, youtube_folder, output_folder, window_size)
    
    